In [27]:
import os
import openai
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS


In [29]:
from dotenv import load_dotenv
load_dotenv()

True

In [30]:
# os.environ["OPENAI_API_KEY"] = "sk-proj-5HIMqYMAxjsJDdPmxzm7T3BlbkFJifATym3GXlDPYSmS2AMu"

In [31]:
def load_and_process_files(file_paths):
    all_documents = []
    for file_path in file_paths:
        loader = TextLoader(file_path)
        documents = loader.load()
        all_documents.extend(documents)
    return all_documents


In [32]:
file_paths = [
    "/home/codezeros/Documents/beautifulsoup/output/output.txt",
    "/home/codezeros/Documents/beautifulsoup/output/ai_for_ecommerce.txt", 
    "/home/codezeros/Documents/beautifulsoup/output/ai_ticketing.txt"
    ]

raw_data = load_and_process_files(file_paths)

print("Loaded document content:", raw_data)

Loaded document content: [Document(page_content='Introduction of Static Websites: Static Websites are the best options for you to start a website in the easiest way. I explained many myths about static websites and how they work. Above all static websites are highly dynamic. Static websites are highly recommended and are suitable for the biggest projects. The codes that are needed to start the static websites don’t change and you can run the codes anywhere you want.\nDetails About Static Websites: Jekyll or Hugo\xa0is the best option to develop static websites. Using these two options the\xa0Forestry CMS\xa0provides the best static websites.\nExamples of Static Websites: Websites that contain stable contents that are displayed using web pages are known as the static website. Here all pages are encrypted using HTML and those WebPages displays provide the same information to all visitors. Dynamic websites require much web design but the static website doesn’t require any web design. That

In [33]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)  
documents = text_splitter.split_documents(raw_data)
print(f"Number of documents: {len(documents)}")

Number of documents: 3


In [34]:
embedding = OpenAIEmbeddings()
db = FAISS.from_documents(documents, embedding)


In [35]:
def get_recommendations(query, db, k=3):
    # Perform a similarity search
    docs = db.similarity_search(query, k=k)  # Retrieve top k matches

    # Verify that the documents are retrieved
    print(f"Retrieved {len(docs)} documents for query: {query}")

    # Extract the best matching lines or sentences
    relevant_sentences = []
    for doc in docs:
        content = doc.page_content
        sentences = content.split('.')  # Split by period to get individual sentences
        for sentence in sentences:
            if query.lower() in sentence.lower():
                relevant_sentences.append(sentence.strip())

    # Combine the relevant sentences or pick the best one based on relevance
    if relevant_sentences:
        # Example heuristic: Choose the sentence that is most relevant to the query
        best_sentence = max(relevant_sentences, key=lambda s: len(s))  # Or use a more advanced scoring method
    else:
        best_sentence = "No relevant sentence found."

    return best_sentence


In [26]:
# def chatbot():
#     print("Welcome to the Personalized Content Recommendation System!")
#     print("Chatbot: How can I assist you?")
#     while True:
#         user_input = input("You: ")
#         if user_input.lower() in ['exit']:
#             print("Chatbot: Goodbye!")
#             break
#         elif user_input.lower() == 'hello':
#             print("Chatbot: How can I help you?")
            
#         else:
#             response = get_recommendations(user_input, db)
#             print(f"Chatbot: {response}.")

# # Run the chatbot
# if __name__ == "__main__":
#     chatbot()


In [17]:
# def get_recommendations(query, db, k=3):
#     # Perform a similarity search
#     docs = db.similarity_search(query, k=k)  # Retrieve top k matches

#     # Verify that the documents are retrieved
#     print(f"Retrieved {len(docs)} documents for query: {query}")

#     # Extract the best matching lines or sentences
#     relevant_sentences = []
#     for doc in docs:
#         content = doc.page_content
#         sentences = content.split('.')  # Split by period to get individual sentences
#         for sentence in sentences:
#             if query.lower() in sentence.lower():
#                 relevant_sentences.append(sentence.strip())

#     # Combine the relevant sentences or pick the best one based on relevance
#     if relevant_sentences:
#         # Example heuristic: Choose the sentence that is most relevant to the query
#         best_sentence = max(relevant_sentences, key=lambda s: len(s))  # Or use a more advanced scoring method
#     else:
#         best_sentence = "No relevant sentence found."

#     return best_sentence


# def chatbot():
#     print("Welcome to the Personalized Content Recommendation System!")
#     print("Chatbot: How can I assist you?")
#     while True:
#         user_input = input("You: ")
#         if user_input.lower() in ['exit']:
#             print("Chatbot: Thank you Goodbye!")
#             break
#         elif user_input.lower() == 'hello':
#             print("You: Hello")
#             print("Chatbot: How can I help you?")
#         else:
#             response = get_recommendations(user_input, db)  # Pass an empty dict or the actual db
#             print(f"You: {user_input}")
#             print(f"Chatbot: {response}")

# # Run the chatbot
# if __name__ == "__main__":
#     chatbot()



In [39]:


def get_recommendations(query, db, k=3):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": query}
        ]
    )
    # Extract the response content
    best_sentence = response['choices'][0]['message']['content']
    return best_sentence

def chatbot():
    print("Welcome to Customer Support Chatbot!")
    print("Chatbot: Hello! How can I assist you?")
    while True:
        user_input = input("User: ")
        if user_input.lower() in ['exit']:
            print("User: Thank you. Goodbye!")
            break
        elif user_input.lower() == 'hello':
            print("User: Hello")
            print("Chatbot: How can I help you?")
        else:
            response = get_recommendations(user_input, db={})  # db is not used in the current implementation
            print(f"User: {user_input}")
            print(f"Chatbot: {response}")

# Run the chatbot
if __name__ == "__main__":
    chatbot()




Welcome to Customer Support Chatbot!
Chatbot: Hello! How can I assist you?
User: hii
Chatbot: Hello! How can I help you today?
User: what is webclues
Chatbot: WebClues is a digital solutions company that provides services in website and mobile app development, digital marketing, and other technology solutions. They work with businesses of various sizes to help them enhance their online presence and drive growth through digital means.
User: what is webclues
Chatbot: WebClues is a global software development company that specializes in providing web and mobile app development services. They offer a wide range of services such as custom software development, e-commerce development, and digital marketing solutions. WebClues has a team of experienced developers and designers who work with businesses to create unique and innovative digital solutions to meet their specific needs.
User: give ve brief
Chatbot: I'm here to help! How can I assist you today?
User: give me brief
Chatbot: Of course!

In [48]:
from langchain.memory import ChatMessageHistory
from langchain_core.memory import AIMessage, UserMessage
import json

chat_history = ChatMessageHistory()


chat_history.add_user_message("what is webclues")
chat_history.add_ai_message("WebClues is a software development company that offers services in website development, mobile app development, e-commerce solutions, digital marketing, and more. They provide custom solutions tailored to meet the specific needs of their clients. With a team of experienced developers and designers, WebClues aims to deliver high-quality digital solutions to help businesses achieve their goals.")
chat_history.add_user_message("Thank you")


print(chat_history.messages)

ImportError: cannot import name 'AIMessage' from 'langchain_core.memory' (/home/codezeros/Documents/beautifulsoup/soup/lib/python3.8/site-packages/langchain_core/memory.py)

In [ ]:
def serialize_chat_history(history):
    serialized_messages = []
    for message in history.messages:
        if isinstance(message, UserMessage):
            serialized_messages.append({"role": "user", "content": message.content})
        elif isinstance(message, AIMessage):
            serialized_messages.append({"role": "assistant", "content": message.content})
    return serialized_messages

# Serialize the chat history
serialized_history = serialize_chat_history(chat_history)

# Save the serialized chat history to a file
with open("chat_history.json", "w") as f:
    json.dump(serialized_history, f, indent=4)

print("Chat history saved to chat_history.json")

In [43]:
import openai
from langchain.memory import ChatMessageHistory, AIMessage, UserMessage

# Initialize the chat history
chat_history = ChatMessageHistory()

def get_recommendations(history, k=3):
    # Prepare the messages list for the OpenAI API
    messages = [{"role": "system", "content": "You are a helpful assistant."}]
    
    # Convert the history messages to the required format
    for message in history.messages:
        if isinstance(message, UserMessage):
            messages.append({"role": "user", "content": message.content})
        elif isinstance(message, AIMessage):
            messages.append({"role": "assistant", "content": message.content})
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )
    # Extract the response content
    best_sentence = response['choices'][0]['message']['content']
    return best_sentence

def chatbot():
    print("Welcome to Customer Support Chatbot!")
    print("Chatbot: Hello! How can I assist you?")
    chat_history.add_ai_message("Hello! How can I assist you?")
    
    while True:
        user_input = input("User: ")
        
        if user_input.lower() in ['exit']:
            print("User: Thank you. Goodbye!")
            chat_history.add_user_message("Thank you. Goodbye!")
            break
        elif user_input.lower() == 'hello':
            print("User: Hello")
            print("Chatbot: How can I help you?")
            chat_history.add_user_message("Hello")
            chat_history.add_ai_message("How can I help you?")
        else:
            chat_history.add_user_message(user_input)
            response = get_recommendations(chat_history)
            print(f"Chatbot: {response}")
            chat_history.add_ai_message(response)

# Run the chatbot
if __name__ == "__main__":
    chatbot()


ImportError: cannot import name 'AIMessage' from 'langchain.memory' (/home/codezeros/Documents/beautifulsoup/soup/lib/python3.8/site-packages/langchain/memory/__init__.py)

In [ ]:
import openai

openai.api_key = "sk-proj-5HIMqYMAxjsJDdPmxzm7T3BlbkFJifATym3GXlDPYSmS2AMu"
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",  
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Tell me pricing for ecommerce."}
  ]
)
print(response['choices'][0]['message']['content'])
